In [1]:
import numpy as np

def is_close_boxes(box1:dict = None, box2:dict = None, min_distance:int = 0, log_view:bool = False):
    """
    두 박스의 거리가 좁은지 확인합니다.
    
    Args:
        box1: 첫 번째 박스
        box2: 두 번째 박스
        min_distance: 최소 거리
    
    Returns:
        True if 두 박스의 거리가 좁으면, False otherwise
    """
    
    box1_center_x = (box1["x"] + box1["w"]) / 2
    box1_center_y = (box1["y"] + box1["h"]) / 2

    
    box2_center_x = (box2["x"] + box2["w"]) / 2
    box2_center_y = (box2["y"] + box2["h"]) / 2
    
    # 두 박스의 중심점 간의 거리를 계산합니다.
    distance = np.sqrt(
        ((box1_center_x - box2_center_x)**2) +
        ((box1_center_y - box2_center_y)**2)
    )

    if log_view:
        print(distance, distance <= min_distance)

    return distance <= min_distance

In [2]:
def merge_boxes(box1, box2):
    """
    두 박스를 하나의 박스로 합칩니다.
    
    Args:
        box1: 첫 번째 박스
        box2: 두 번째 박스
    
    Returns:
        합쳐진 박스
    """
    
    return {
        "x": min(box1["x"], box2["x"]), # X
        "y": min(box1["y"], box2["y"]), # Y
        "w": max(box1["w"], box2["w"]), # W
        "h": max(box1["h"], box2["h"]), # H
    }

In [3]:
import cv2

image = cv2.imread("../dataset_dir/13.한국어글자체/01.손글씨/01_handwriting_sentence_images/1_sentence/00000003.png")
image2 = image.copy()

# 컨투어 찾기 전 이미지 전처리
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
gray = cv2.bitwise_not(gray) # 객체보다 배경이 밝은 경우 이미지 반전

# threshold 등 다양한 전처리 시도 -> 객체와 배경을 가장 잘 분리하는 전처리 사용
_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

thresh = cv2.erode(thresh, None, iterations=2)
thresh = cv2.dilate(thresh, None, iterations=2)

# 컨투어 찾기
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [4]:
processed_boxes = []
original_boxes = []

for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    
    original_boxes.append({
        "x": x, 
        "y": y, 
        "w": w, 
        "h": h
    })
    processed_boxes.append({
        "x": x, 
        "y": y, 
        "w": (x + w), 
        "h": (y + h)
    })

original_boxes = sorted(original_boxes, key=lambda country: country["x"])
processed_boxes = sorted(processed_boxes, key=lambda country: country["x"])

In [5]:
merged_boxes = []
for i in range(len(processed_boxes)):
    merged_box = processed_boxes[i]
    
    for j in range(i+1, len(processed_boxes)):
        if is_close_boxes(merged_box, processed_boxes[j], 60, True):
            merged_box = merge_boxes(merged_box, processed_boxes[j])
            merged_boxes.append(merged_box)

64.01757571167468 False
41.07614879708174 True
70.83784299369935 False
63.12289600454022 False
71.97569034055873 False
104.20292702222908 False
167.81090548590697 False
151.43728074684913 False
222.1424092783726 False
278.21754078418564 False
334.68081809389673 False
383.45338699768973 False
377.50298012068725 False
474.0 False
456.835309493476 False
482.256415198388 False
493.1029304313654 False
560.788953172225 False
610.0331958180636 False
643.6536335638913 False
755.5761047042184 False
761.086887286859 False
802.6183713322291 False
854.8230811109396 False
901.8748527373407 False
931.8627849635375 False
1031.7563908210116 False
1083.4726577076137 False
1142.0858111367988 False
1190.3416526359144 False
1191.685046478305 False
1322.3071693067386 False
1410.5468797597619 False
1424.2269833141065 False
1474.3693567081486 False
1537.6434079460687 False
1591.3422007852364 False
1702.676275749445 False
1721.1567331303677 False
1708.5237048399417 False
1791.2596824581299 False
1831.70439754

In [6]:
# 거리가 좁은 박스를 하나의 박스로 합칩니다.
# merged_boxes = []
# for i in range(len(processed_boxes)):
#     if is_close_boxes(processed_boxes[i], processed_boxes[i+1], 50, True):
#         merged_box = merge_boxes(processed_boxes[i], processed_boxes[i+1])
#         merged_boxes.append(merged_box)

In [7]:
# 합쳐진 박스를 표기합니다.
for box in processed_boxes:
    cv2.rectangle(image, (box["x"], box["y"]), (box["w"], box["h"]), (0, 255, 0), 2)

# 결과 이미지 출력
cv2.imwrite(f'./image/None_processed_boxes.jpg', image)

True

In [8]:
# 합쳐진 박스를 표기합니다.
for box in merged_boxes:
    cv2.rectangle(image2, (box["x"], box["y"]), (box["w"], box["h"]), (0, 255, 0), 2)

# 결과 이미지 출력
cv2.imwrite(f'./image/None_merged_boxes.jpg', image2)

True